Clasifica una imagen entre tres consolas de videojuegos. 

Estructura de carpetas del proyecto

```
Clasificador/
├── data/                 Almacena los datos necesario del proyecto
|   ├── entrenamiento/    Almacena las imagenes de entrenamiento
|   |   ├── Opcion1/      Almacena imágenes de la opcion1
|   |   ├── Opcion2/      Almacena imágenes de la opcion2
|   |   └── Opcion3/      Almacena imágenes de la opcion3
|   └── validacion/       Almacena las imagenes de validación
|       ├── Opcion1/      Almacena imágenes de la opcion1
|       ├── Opcion2/      Almacena imágenes de la opcion2
|       └── Opcion3/      Almacena imágenes de la opcion3
├── cnn.ipynb             
├── entrenar.ipynb        Contendrá la Red nerual y se entrenará
└── predecir.ipynb        Cargar el modelo y generar la predicción

```

In [1]:
# Libreria para movernos en el sistema operativo
import sys
import os

import tensorflow as tf

from tensorflow import keras

# Ayuda a preprocesar las imagenes que le daremos al algoritmo
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

# Optimizador con el que entrenaremos al algoritmo
from tensorflow.python.keras import optimizers

# Permite hacer redes neurales secuenciales
from tensorflow.python.keras.models import Sequential

# 
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation

# Capas en las que trabajaremos las capas de convoluciones y maxPooling
from tensorflow.python.keras.layers import Convolution2D, MaxPooling2D

# Matar sesión existente de keras para entrenar desde 0
from tensorflow.python.keras import backend as k

In [2]:
# Matar sesión existente de keras para entrenar desde 0
k.clear_session()

# Directorios de los datos a usar
DATA_ENTRENAMIENTO = './data/entrenamiento'
DATA_VALIDACION = './data/validacion'

In [3]:
# Parámetros

EPOCAS = 20                 # Numero de veces que iteraremos en el set de datos
LONGITUD, ALTURA = 100,100  # Dimensiones de la imagenes a procesar
BATCH_SIZE = 32             # Número de imágenes a procesar en cada iteración
PASOS = 90                  # Veces que se procesará los datos en una época
PASOS_VALIDACION = 300      # A final de cada época se correrá esta cantidad
FILTROS_CONV_1 = 32         # Número de filtros que hará en cada imagen en la primera convolucion
FILTROS_CONV_2 = 64         # Número de filtros que hará en cada imagen en la segunda convolucion
TAMANO_FILTRO_1 = (3,3)     # Tamaño del filtro que estaremos usando en la primera convolución
TAMANO_FILTRO_2 = (2,2)     # Tamaño del filtro que estaremos usando en la segunda convolución
TAMANO_POOL = (2,2)         # Tamaño del filtro que se usará en el MaxPooling
CLASES = 3                  # Número de elementos a clasificar
LR = 0.0005                 # Learning Rate: Qué tan grande son los ajustes de la red neural para buscar una solución

In [4]:
# Pre-procesamiento de imágenes

entrenamiento_datagen = ImageDataGenerator(
    rescale = 1./255,       # Reescala cada uno de nuestros píxeles en vez que tengan un rango de 1 a 255, tenga uno de 0 a 1
    shear_range = 0.3,      # Genera imagenes inclinadas
    zoom_range = 0.3,       # Genera imágenes con zoom, esto para que el algoritmo aprenda que puede aparecer secciones del elemento
    horizontal_flip = True  # Genera imágenes invertidas
)

validacion_datagen = ImageDataGenerator(
    rescale = 1./255        # Reescala cada uno de nuestros píxeles en vez que tengan un rango de 1 a 255, tenga uno de 0 a 1
)


entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    DATA_ENTRENAMIENTO,                 # Entrar al directorio de las imagtenes de entrenamiento
    target_size = (ALTURA,LONGITUD),    # Analizará la imagen con un ancho y altura especificada
    batch_size = BATCH_SIZE,            # Tamaño ya definido del batch
    class_mode = 'categorical'          # El tipo de clasificacion que haremos es categorical
)

validacion_generador = validacion_datagen.flow_from_directory(
    DATA_VALIDACION,                 # Entrar al directorio de las imagtenes de entrenamiento
    target_size = (ALTURA,LONGITUD),    # Analizará la imagen con un ancho y altura especificada
    batch_size = BATCH_SIZE,            # Tamaño ya definido del batch
    class_mode = 'categorical'          # El tipo de clasificacion que haremos es categorical
)

Found 297 images belonging to 3 classes.
Found 91 images belonging to 3 classes.


In [5]:
# Crear la red CNN

cnn = Sequential()

# Primera convolución
cnn.add(Convolution2D(FILTROS_CONV_1, TAMANO_FILTRO_1, padding="same", input_shape=(LONGITUD,ALTURA,3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=TAMANO_POOL))

# Segunda convolución
cnn.add(Convolution2D(FILTROS_CONV_2, TAMANO_FILTRO_2, padding='same', activation='relu'))
cnn.add(MaxPooling2D(pool_size=TAMANO_POOL))

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
cnn.add(Flatten())                           # La imagen restante de las convoluciones, la volvemos unidimensional

In [7]:
cnn.add(Dense(256, activation='relu'))       # 256 Neuronas en esta capa

In [8]:
keep_prob = 0.5
cnn.add(Dropout(rate=1-keep_prob))                        # A cada paso se apaga el 50% de las neuronas a la capa densa, esto para evitar sobre-ajustar

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
cnn.add(Dense(CLASES, activation='softmax')) # Esta capa solo va a tener 3 neuronas con una activación de SoftMax 

In [10]:
cnn.compile(loss='categorical_crossentropy',                   # La función de pérdida sea categorical_crossentropy
            optimizer="adam", # Se utiliza el optimizador Adam con el learning rate ya especificado
            metrics=['accuracy']                               # La métrica a optimizar es el % de qué tan bien está siendo entrenada la red neuronal
)

In [11]:
cnn.fit(                                    # Para entrenar
    entrenamiento_generador,                # Usar las imagenes ya preprocesadas de entrenamiento
    steps_per_epoch = PASOS,                # Determinar la cantidad de pasos que tendrá las épocas
    epochs = EPOCAS,                        # Cantidad de épocas 
    validation_data = validacion_generador, # Usar las imagenes ya preprocesadas de validación
    validation_steps = PASOS_VALIDACION     # Qué tantos pasos realizar de validación en cada época
)

DIR_MODEL = './modelo/'

if not os.path.exists(DIR_MODEL):  # Si no existe la carpeta modelo, la crea
    os.mkdir(dir)

cnn.save('./modelo/modelo.h5')          # Guarda la estructura del modelo
cnn.save_weights('./modelo/pesos.h5')   # Guarda los valores de los pesos del modelo ya entrenado


Instructions for updating:
Use tf.cast instead.
Epoch 1/20
10/10 [==============================] - 19s 2s/step - loss: 2.6417 - acc: 0.3468 - val_loss: 1.2198 - val_acc: 0.3297
Epoch 2/20
10/10 [==============================] - 14s 1s/step - loss: 1.0623 - acc: 0.5152 - val_loss: 0.8808 - val_acc: 0.6593
Epoch 3/20
10/10 [==============================] - 14s 1s/step - loss: 0.8474 - acc: 0.6364 - val_loss: 0.7060 - val_acc: 0.7253
Epoch 4/20
10/10 [==============================] - 14s 1s/step - loss: 0.7888 - acc: 0.6667 - val_loss: 0.6382 - val_acc: 0.7363
Epoch 5/20
10/10 [==============================] - 15s 1s/step - loss: 0.6669 - acc: 0.7508 - val_loss: 0.5643 - val_acc: 0.7802
Epoch 6/20
10/10 [==============================] - 15s 2s/step - loss: 0.6059 - acc: 0.7778 - val_loss: 0.6181 - val_acc: 0.7912
Epoch 7/20
10/10 [==============================] - 14s 1s/step - loss: 0.5737 - acc: 0.7677 - val_loss: 0.4651 - val_acc: 0.8242
Epoch 8/20
10/10 [========================